In [23]:
import pandas as pd
import json
from pathlib import Path
import ast

In [24]:
def read_jsonl_file(file_path):
    """
    JSON Lines 형식의 파일을 읽는 함수
    """
    data = []
    try:
        # 파일 존재 여부 확인
        if not Path(file_path).is_file():
            print(f"파일이 존재하지 않습니다: {file_path}")
            return None

        # JSON Lines 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_number, line in enumerate(f, start=1):
                try:
                    json_obj = json.loads(line.strip())
                    data.append(json_obj)
                except json.JSONDecodeError as e:
                    print(f"JSON 파싱 오류 (라인 {line_number}, 무시됨): {str(e)}")
                    continue
        return data
    except Exception as e:
        print(f"파일 읽기 오류 ({file_path}): {str(e)}")
        return None


def compare_with_reference(reference_path, comparison_paths):
    """
    기준 파일과 다른 파일들을 비교하는 함수

    Parameters:
    reference_path (str): 기준 파일 경로
    comparison_paths (list): 비교할 파일들의 경로 리스트

    Returns:
    dict: 비교 결과
    """
    # 기준 파일 읽기
    reference_data = read_jsonl_file(reference_path)
    if not reference_data:
        print("기준 파일을 읽을 수 없습니다.")
        return None

    reference_name = Path(reference_path).stem
    reference_topk = [item['topk'] for item in reference_data if 'topk' in item]

    # 비교 결과를 저장할 딕셔너리
    comparison_results = {}

    # 각 비교 파일에 대해 비교 수행
    for comp_path in comparison_paths:
        comp_data = read_jsonl_file(comp_path)
        if not comp_data:
            continue

        comp_name = Path(comp_path).stem
        comp_topk = [item['topk'] for item in comp_data if 'topk' in item]

        matches = []
        for idx, (ref_vals, comp_vals) in enumerate(zip(reference_topk, comp_topk)):
            set_ref = set(ref_vals)
            set_comp = set(comp_vals)

            # Intersection and differences
            intersection = set_ref & set_comp
            unique_to_ref = set_ref - set_comp
            unique_to_comp = set_comp - set_ref

            # ZeroDivisionError 방지
            union_length = len(set_ref.union(set_comp))
            match_percentage = 0 if union_length == 0 else round(len(intersection) / union_length * 100, 2)

            matches.append({
                'row': idx + 1,
                'common_values': sorted(list(intersection)),
                'unique_to_reference': sorted(list(unique_to_ref)),
                'unique_to_comparison': sorted(list(unique_to_comp)),
                'match_percentage': match_percentage
            })

        comparison_results[comp_name] = matches

    return reference_name, comparison_results


def save_comparison_results(reference_name, results, output_file):
    """
    비교 결과를 CSV 파일로 저장
    """
    rows = []
    for comp_name, matches in results.items():
        for match in matches:
            row = {
                'Reference File': reference_name,
                'Comparison File': comp_name,
                'Row': match['row'],
                'Match Percentage': f"{match['match_percentage']}%",
                'Common Values Count': len(match['common_values']),
                'Common Values': ', '.join(match['common_values']),
                'Unique to Reference Count': len(match['unique_to_reference']),
                'Unique to Reference': ', '.join(match['unique_to_reference']),
                'Unique to Comparison Count': len(match['unique_to_comparison']),
                'Unique to Comparison': ', '.join(match['unique_to_comparison'])
            }
            rows.append(row)

    df = pd.DataFrame(rows)
    df.to_csv(output_file, index=False)
    print(f"\n결과가 저장됨: {output_file}")

In [25]:
def main():
    # 기준 파일과 비교할 파일들 지정
    reference_file = '/home/0.8856.csv'  # 기준 파일 경로
    comparison_files = [        
        '/home/qurey수정_final.csv'
    ]

    # 비교 수행
    reference_name, results = compare_with_reference(reference_file, comparison_files)

    if results:
        print(f"\n=== 기준 파일: {reference_name} ===")
        
        # 비교 결과 요약 출력
        summary = []
        for comp_name, matches in results.items():
            total_matches = len(matches)
            exact_matches = sum(1 for m in matches if m['match_percentage'] == 100)
            avg_match_percentage = sum(m['match_percentage'] for m in matches) / total_matches
            
            # 파일별 결과 출력
            print(f"\n[비교 파일: {comp_name}]")
            print(f" - 총 비교 행 수: {total_matches}")
            print(f" - 완전 일치 행 수: {exact_matches}")
            print(f" - 평균 일치율: {avg_match_percentage:.2f}%")
            
        # 결과 저장
        output_file = '/home/IR/data/reference_comparison_results.csv'
        save_comparison_results(reference_name, results, output_file)       
    else:
        print("비교 수행 중 오류가 발생했습니다. 결과가 없습니다.")

if __name__ == "__main__":
    main()


=== 기준 파일: 0.8856 ===

[비교 파일: qurey수정_final]
 - 총 비교 행 수: 220
 - 완전 일치 행 수: 184
 - 평균 일치율: 86.73%

결과가 저장됨: /home/IR/data/reference_comparison_results.csv
